In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re
import os

import pandas as pd
import numpy as np

import openpyxl
from openpyxl.styles import PatternFill
from openpyxl import Workbook
from random import uniform

In [3]:
df = pd.read_excel(r'C:\Users\NT550\asac\SIH\SIH\code2\result\꿀_스틱_지식인(16.08.01-23.08.16).xlsx')
df.head()

,제목,질문,답변
0,가독성 필요합니다..,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,가독성 부탁드립니다 ㅠㅠ,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,선생님 선물 추천,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,NaN,저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,NaN,다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...


In [19]:
# clean_filename 함수는 파일 이름에 사용할 수 없는 문자를 '_'로 대체합니다.
def clean_filename(filename):
    return re.sub(r'[\/:*?"<>|]', '_', filename)

# 크롬 드라이버 버전에 맞는 경로로 수정해주세요.
path = "C:/chromedriver.exe"
driver = webdriver.Chrome(executable_path=path)

# 네이버 지식인 크롤링
# keyword에 크롤링하고 싶은 단어 선택. space 는 + 로 치환
def get_keyword(text):
    return text.replace(" ", "%20")

# 정렬 방식 선택
# 1: 추천순
# 2: 최신순
# 기타: 정확도 순
def sort_kind(index):
    # 추천
    if index == 1:
        return 'vcount'
    # 최신순
    elif index == 2:
        return 'date'
    # 정확도
    else:
        return 'none'

keyword = '"꿀 스틱"'
cleaned_keyword = clean_filename(keyword.replace(' ', '_'))

driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

page_index = 1
# 크롤링 시작 일자
f = '16.08.01'
# 크롤링 종료 일자
t = '2023.08.19'
period_txt = "&period=" + f + ".%7C" + t + "."

_sort_kind = sort_kind(2)
date = str(datetime.now()).replace('.', '_')
date = date.replace(' ', '_')

# URL 저장
cleaned_date = clean_filename(date)
url_filename = f"result/url_list_{cleaned_keyword}_{cleaned_date}.txt"

# result 디렉토리가 존재하지 않으면 생성
result_dir = "result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

f = open(url_filename, 'w')

page_url = []

while True:
    time.sleep(uniform(0.01, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.find_all('a', class_="_nclicks:kin.txt _searchListTitleAnchor")
    for tag in tags:
        url = str(tag).split(' ')[3]
        url = url.replace('href=', "")
        url = url.replace('"', "")
        url = url.replace('amp;', '')
        page_url.append(url)
        f.write(url + "\n")

    post_number = driver.find_element_by_class_name('number').text
    post_number = str(post_number).replace("(", "")
    post_number = str(post_number).replace(")", "")
    
    current_number = post_number.split('/')[0].split('-')[1]
    current_number = current_number.replace(',', '')
    total_number = post_number.split('/')[1]
    total_number = total_number.replace(',', '')

    if int(current_number) == int(total_number):
        break
    else:
        page_index += 1

filename = f"result/{cleaned_keyword}_{cleaned_date}_crawling_result.xlsx"
wb = Workbook()
sheet = wb.active
sheet.append(['제목', '아이디', '날짜', '질문', '답변'])

for j in range(1, 6):
    sheet.cell(row=1, column=j).fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')

for i in page_url:
    driver.get(i)
    title = driver.find_element_by_class_name('title').text
    id = driver.find_element_by_class_name('c-userinfo__author').text
    date = driver.find_element_by_class_name('c-userinfo__info').text
    try:
        question_txt = driver.find_element_by_class_name('c-heading__content').text
        
    except:
        question_txt = ""

    # 답변 리스트
    answer_list = driver.find_elements_by_class_name("se-main-container")
    
    for n, answer in enumerate(answer_list):
        texts = answer.find_elements_by_tag_name('span')
        t = ""
        for i in texts:
            t += i.text

        if n == 0:
            sheet.append([title, id, date, question_txt, t])
        else:
            sheet.append(["", "", t])

    wb.save(filename)

driver.quit()  # 작업이 끝난 후 브라우저를 닫습니다.

C:\Users\NT550\AppData\Local\Temp\ipykernel_8140\2461931632.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path)
C:\Users\NT550\AppData\Local\Temp\ipykernel_8140\2461931632.py:74: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  post_number = driver.find_element_by_class_name('number').text
C:\Users\NT550\AppData\Local\Temp\ipykernel_8140\2461931632.py:98: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  title = driver.find_element_by_class_name('title').text
C:\Users\NT550\AppData\Local\Temp\ipykernel_8140\2461931632.py:99: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  id = driver.find_element_by_class_name('c-userinfo__author').text
C:\Users\NT550\AppData\Local\Te

In [20]:
df = pd.read_excel(r'C:\Users\NT550\asac\SIH\SIH\code2\result\_꿀_스틱__2023-08-20_00_29_50_308957_crawling_result.xlsx')
df.head()

,제목,아이디,날짜,질문,답변
0,가독성 필요합니다..,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,가독성 부탁드립니다 ㅠㅠ,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,선생님 선물 추천,닉네임\n비공개,작성일\n2023.05.30,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,닉네임\n비공개,작성일\n2023.05.30,"하루에 스틱꿀 1포씩 먹으면 어디에 좋나요? 또, 언제... 스틱꿀 1포씩 먹는 것...",저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,NaN,NaN,다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...,NaN,NaN


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      84 non-null     object
 1   아이디     84 non-null     object
 2   날짜      271 non-null    object
 3   질문      84 non-null     object
 4   답변      84 non-null     object
dtypes: object(5)
memory usage: 10.7+ KB


In [5]:
# 모든 컬럼에 '꿀'이라는 단어가 없는 행을 필터링합니다.
filtered_df = df[~(df['제목'].str.contains('꿀', na=False) &
                  df['질문'].str.contains('꿀', na=False) &
                  df['답변'].str.contains('꿀', na=False))]

# 결과를 출력합니다.
filtered_df.head()

,제목,질문,답변
0,가독성 필요합니다..,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,가독성 부탁드립니다 ㅠㅠ,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,선생님 선물 추천,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,NaN,저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,NaN,다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...


In [6]:
filtered_df.to_excel('꿀_스틱.xlsx')

In [2]:
df = pd.read_excel(r'C:\Users\NT550\asac\SIH\SIH\code2\result\_꿀_스틱__2023-08-20_00_29_50_308957_crawling_result.xlsx')
df.head()

,제목,아이디,날짜,질문,답변
0,가독성 필요합니다..,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,가독성 부탁드립니다 ㅠㅠ,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,선생님 선물 추천,닉네임\n비공개,작성일\n2023.05.30,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,닉네임\n비공개,작성일\n2023.05.30,"하루에 스틱꿀 1포씩 먹으면 어디에 좋나요? 또, 언제... 스틱꿀 1포씩 먹는 것...",저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,NaN,NaN,다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...,NaN,NaN


In [3]:
# 리뷰 출력시 ... 없이 전부 나오게 설정
""" pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
 """
for index, row in df.iterrows():
    if pd.isna(row['제목']):
        row['답변'] = row['날짜']
        row['날짜'] = np.nan

df


,제목,아이디,날짜,질문,답변
0,가독성 필요합니다..,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,가독성 부탁드립니다 ㅠㅠ,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,선생님 선물 추천,닉네임\n비공개,작성일\n2023.05.30,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,닉네임\n비공개,작성일\n2023.05.30,"하루에 스틱꿀 1포씩 먹으면 어디에 좋나요? 또, 언제... 스틱꿀 1포씩 먹는 것...",저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,NaN,NaN,NaN,NaN,다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...
...,...,...,...,...,...
266,면역력높이는영양제 정말로 괜찮은걸로 좀..!,닉네임\n비공개,작성일\n2019.11.12,30대 중반 동갑내기 부부입니다. 면역력높이는영양제 이제 더는 미룰 수 없어서요.....,요새 환절기에 오늘 비오고 나면 영하권으로 내려간대요. 나이들면 더 챙겨먹어야겠더라...
267,NaN,NaN,NaN,NaN,우리 몸에서 면역을 담당하는 것은 소화기관인 장입니다.장속에 있는 세균들이 우리 몸...
268,NaN,NaN,NaN,NaN,면역력높여주는데에는 면역글로불린 성분이 풍부한 뉴질랜드초유가 원료도 안심하여 먹을 ...
269,NaN,NaN,NaN,NaN,"혈액순환이 저하되면 체온이 저하되고몸의 자연치유력이 저하되면서 몸속에 염증, 물혹 ..."


In [7]:
df = pd.read_excel(r'C:\Users\NT550\asac\SIH\SIH\code2\code\네이버지식인.xlsx')
df.head()

,Unnamed: 0,제목,아이디,날짜,질문,답변
0,0,가독성 필요합니다..,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,"선호하는 세트 구성품을 다량으로 개별 구매할 수 있도록 하여, 고객 개인이 구성품을..."
1,1,가독성 부탁드립니다 ㅠㅠ,닉네임\nnate****,작성일\n2023.07.07,업무용인데 제가 많이 부족해서 요약이 어렵습니다...\n\n-내용\n세트를 한 번 ...,기안서1. 작성사유기존 A세트 시식 후 좋은 반응을 보임. (대략적인 추측 자료가 ...
2,2,선생님 선물 추천,닉네임\n비공개,작성일\n2023.05.30,티 세트랑 수제청이랑 꿀 세트 중에서 고민하고 있는데 뭐가 좋을까요?\n남자 선생님...,꿀 스틱같은건 어떨까요?꿀 차로도 드시기 편하고 꿀이 목 보호에 좋거든요~^^
3,3,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,닉네임\n비공개,작성일\n2023.05.30,"하루에 스틱꿀 1포씩 먹으면 어디에 좋나요? 또, 언제... 스틱꿀 1포씩 먹는 것...",저는 안 해요. 처음에는 했었는데 운동만 적는 게 괜찮습니다. 스트레스 받지 않고 ...
4,4,다이어트하는데 다이어트일지 중요하죠?\n\n뭐뭐먹었는지 운동은 뭐했는지\n\n다이어...,닉네임\n비공개,작성일\n2023.05.30,"하루에 스틱꿀 1포씩 먹으면 어디에 좋나요? 또, 언제... 스틱꿀 1포씩 먹는 것...",다이어트 일지 쓰면 아무래도 조금 더 체계적으로 체크할 수 있어서 좋은 거 같더라구...
